## Notebook to create feature pandas for Model Input:

In [7]:
import pandas as pd
import numpy as np, time
from geopy.distance import great_circle
import math
import seaborn as sns
from scipy import spatial
import matplotlib.pyplot as plt
import datetime
from sqlalchemy import create_engine

import config
import config1 


In [8]:
# create sqlalchemy engine
engine = create_engine("mysql+mysqlconnector://{user}:{password}@{host}/{dbname}"
                       .format(user=config.db_user,
                               password=config.db_pass,
                               dbname=config.db_name,
                               host=config.db_host))

In [4]:
start_time = time.time()
x = 5 + 5*3
print(time.time()-start_time)
print(x)

6.67572021484375e-05
20


In [5]:
# Expanding number of columns:
pd.set_option('display.max_columns', 40)

In [6]:
NASA_M6 = pd.read_csv('/Users/AlfHaugen/Python/Wildfire_Data/FireExports/NASA-to-ClustserRef/NASA_M6_FullData_ClusterRef_May30.csv')

/Users/AlfHaugen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
emdata = pd.read_csv('/Users/AlfHaugen/Python/Wildfire_Data/FireExports/Emissions Cluster Data/emissions_03_15_v5.24.csv')

In [8]:
print(NASA_M6.shape)
NASA_M6.head()

(2159441, 21)


,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,datetime,doy,month,year,target_clusterref
0,0,38.8142,-93.5539,300.8,1.0,1.0,2003-01-01,423,Terra,MODIS,33,6.2,267.0,10.4,N,0,2003-01-01,1,1,2003,2003_1980
1,1,19.3739,-155.1130,318.8,3.1,1.7,2003-01-01,914,Terra,MODIS,97,6.2,288.7,98.4,N,2,2003-01-01,1,1,2003,NaN
2,2,19.3723,-155.1197,316.0,3.1,1.7,2003-01-01,914,Terra,MODIS,90,6.2,288.3,85.0,N,2,2003-01-01,1,1,2003,NaN
3,3,19.3589,-155.1107,325.2,3.1,1.7,2003-01-01,914,Terra,MODIS,100,6.2,292.0,131.8,N,2,2003-01-01,1,1,2003,NaN
4,4,19.3573,-155.1174,319.0,3.1,1.7,2003-01-01,914,Terra,MODIS,97,6.2,290.3,97.3,N,2,2003-01-01,1,1,2003,NaN


In [9]:
#NASA_M6_Model = NASA_M6[(NASA_M6.NASA_M6 != null)]
NASA_M6_Model = NASA_M6[NASA_M6.target_clusterref.notnull()]
print(NASA_M6_Model.shape)
NASA_M6_Model.head()

(1352421, 21)


,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,datetime,doy,month,year,target_clusterref
0,0,38.8142,-93.5539,300.8,1.0,1.0,2003-01-01,423,Terra,MODIS,33,6.2,267.0,10.4,N,0,2003-01-01,1,1,2003,2003_1980
21,21,34.5954,-78.6218,306.5,1.3,1.1,2003-01-01,1624,Terra,MODIS,64,6.2,289.2,11.0,D,0,2003-01-01,1,1,2003,2003_4279
22,22,33.4182,-110.8618,307.6,1.2,1.1,2003-01-01,1803,Terra,MODIS,66,6.2,285.1,10.8,D,2,2003-01-01,1,1,2003,2003_1522
23,23,29.7120,-95.1284,307.2,1.0,1.0,2003-01-01,1934,Aqua,MODIS,65,6.2,294.1,5.6,D,0,2003-01-01,1,1,2003,2003_919
24,24,28.9161,-98.6293,313.3,1.3,1.1,2003-01-01,1934,Aqua,MODIS,69,6.2,297.4,12.0,D,0,2003-01-01,1,1,2003,2003_777


In [10]:
print(emdata.shape)
emdata.head(10)

(5936438, 21)


,Unnamed: 0,id,year,doy,longitude,latitude,covertype,fuelcode,area_burned,prefire_fuel,consumed_fuel,ECO2,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burnday_source,BSEV,cluster_reference
0,0,0,2008,359,-81.0384,25.1958,3,1600,0.0,6220.097576,1999.75772,3347.594423,153.981344,4.499455,23.797117,0.023231,0.082115,3,81,1,2008_0
1,1,1,2008,359,-81.0404,25.1984,3,1600,62500.0,6220.097576,2041.37434,3417.260644,157.185824,4.593092,24.292355,0.022757,0.080441,3,81,2,2008_0
2,2,2,2008,359,-81.0380,25.1981,3,1600,0.0,6220.097576,1999.75772,3347.594423,153.981344,4.499455,23.797117,0.023231,0.082115,3,81,1,2008_0
3,3,4,2008,359,-81.0594,25.2035,3,1600,0.0,6220.097576,1999.75772,3347.594423,153.981344,4.499455,23.797117,0.023231,0.082115,3,81,1,2008_0
4,4,5,2008,359,-81.0570,25.2032,3,1600,62500.0,6220.097576,2041.37434,3417.260644,157.185824,4.593092,24.292355,0.022757,0.080441,3,81,2,2008_0
5,5,6,2008,359,-81.0545,25.2028,3,1600,62500.0,6220.097576,2041.37434,3417.260644,157.185824,4.593092,24.292355,0.022757,0.080441,3,81,2,2008_0
6,6,7,2008,359,-81.0521,25.2025,3,1600,0.0,6220.097576,1999.75772,3347.594423,153.981344,4.499455,23.797117,0.023231,0.082115,3,81,1,2008_0
7,7,8,2008,359,-81.0473,25.2018,3,1600,0.0,6220.097576,1999.75772,3347.594423,153.981344,4.499455,23.797117,0.023231,0.082115,3,81,1,2008_0
8,8,9,2008,359,-81.0449,25.2014,3,1600,62500.0,6220.097576,2041.37434,3417.260644,157.185824,4.593092,24.292355,0.022757,0.080441,3,81,2,2008_0
9,9,10,2008,359,-81.0400,25.2007,3,1600,0.0,6220.097576,1999.75772,3347.594423,153.981344,4.499455,23.797117,0.023231,0.082115,3,81,1,2008_0


In [46]:
emdata.loc[emdata['cluster_reference'] == '2003_1980']

,Unnamed: 0,id,year,doy,longitude,latitude,covertype,fuelcode,area_burned,prefire_fuel,consumed_fuel,ECO2,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burnday_source,BSEV,cluster_reference
5907041,5907041,7205653,2003,81,-93.3088,39.3217,1,1,62500.0,238.967352,222.239637,372.695872,15.556775,0.595602,2.266844,0.000000,0.000000,2,12,2,2003_1980
5907042,5907042,7205654,2003,83,-93.3058,39.3217,3,2700,0.0,2639.337818,1979.738069,3076.512960,263.305163,14.808441,45.138028,0.162671,0.157472,2,12,1,2003_1980


In [11]:
# Set Season by DOY = 0 is Spring, 1 is Summer, 2 is Fall and 3 is Winter:
def get_season(doy):
    """
    convert date to month and day as integer (md), e.g. 4/21 = 421, 11/17 = 1117, etc.
    """
    #m = date.month * 100
    #d = date.day
    #md = m + d
    
    if ((doy >= 80) and (doy <= 172)):
        s = 0  # spring
    elif ((doy > 172) and (doy < 264)):
        s = 1  # summer
    elif ((doy >= 264) and (doy <= 355)):
        s = 2  # fall
    #elif ((doy > 355) and (doy < 80)):
        #s = 3  # winter
    else:
        s = 3
     #   raise IndexError("Invalid date")
    return s

#spring = range(80, 172)
#summer = range(172, 264)
#fall = range(264, 355)

In [12]:
print(get_season(265))

2


In [13]:
#Creating Season per DOY. 
NASA_M6_Model['season'] = NASA_M6_Model.apply(lambda x: get_season(x['doy']),axis=1)

/Users/AlfHaugen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
NASA_M6_Model.head()

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,datetime,doy,month,year,target_clusterref,season
0,0,38.8142,-93.5539,300.8,1.0,1.0,2003-01-01,423,Terra,MODIS,33,6.2,267.0,10.4,N,0,2003-01-01,1,1,2003,2003_1980,3
21,21,34.5954,-78.6218,306.5,1.3,1.1,2003-01-01,1624,Terra,MODIS,64,6.2,289.2,11.0,D,0,2003-01-01,1,1,2003,2003_4279,3
22,22,33.4182,-110.8618,307.6,1.2,1.1,2003-01-01,1803,Terra,MODIS,66,6.2,285.1,10.8,D,2,2003-01-01,1,1,2003,2003_1522,3
23,23,29.7120,-95.1284,307.2,1.0,1.0,2003-01-01,1934,Aqua,MODIS,65,6.2,294.1,5.6,D,0,2003-01-01,1,1,2003,2003_919,3
24,24,28.9161,-98.6293,313.3,1.3,1.1,2003-01-01,1934,Aqua,MODIS,69,6.2,297.4,12.0,D,0,2003-01-01,1,1,2003,2003_777,3


In [15]:
start_time = time.time()

x = '2008_0'
y = emdata.loc[emdata['cluster_reference'] == x, 'covertype'].value_counts().idxmax()
print(y)
print(time.time()-start_time)

3
0.43146491050720215


In [16]:
def get_covertype(target_clusterref):
    covertype = emdata.loc[emdata['cluster_reference'] == target_clusterref, 'covertype'].value_counts().idxmax()
    return covertype

In [17]:
f = get_covertype('2008_0')
f

3

In [25]:
#covertype = emdata.groupby( [ "cluster_reference"] ).value_counts().to_frame(name = 'covertype').reset_index()
#covertype = emdata.groupby( ['cluster_reference', 'covertype'] ).size().to_frame(name = 'covermode').reset_index()
#covertype = emdata.groupby( [ 'cluster_reference', 'covertype'] ).agg(lambda x:x.value_counts().index[0])
covertypedf = emdata.groupby('cluster_reference')['covertype'].agg(pd.Series.mode).to_frame(name = 'covertype').reset_index()
covertypedf['covertype2'] = covertypedf.apply(lambda x: (x['covertype'].max()),axis=1)
covertypedf.head()

,cluster_reference,covertype
0,2003_0,1
1,2003_1,3
2,2003_10,1
3,2003_100,2
4,2003_1000,3


In [49]:
covertypedf['covertype2'] = covertypedf.apply(lambda x: (x['covertype'].max()),axis=1)
covertypedf.head()

,cluster_reference,covertype,covertype2
0,2003_0,1,1
1,2003_1,3,3
2,2003_10,1,1
3,2003_100,2,2
4,2003_1000,3,3


In [50]:
covertypedf.loc[covertypedf['cluster_reference'] == '2003_1980']

,cluster_reference,covertype,covertype2
1090,2003_1980,"[1, 3]",3


In [52]:
fuelcodedf = emdata.groupby('cluster_reference')['fuelcode'].agg(pd.Series.mode).to_frame(name = 'fuelcode').reset_index()
fuelcodedf.head()

,cluster_reference,fuelcode
0,2003_0,1
1,2003_1,1600
2,2003_10,1
3,2003_100,2
4,2003_1000,2700


In [54]:
fuelcodedf['fuelcode2'] = fuelcodedf.apply(lambda x: (x['fuelcode'].max()),axis=1)

In [22]:
prefirefuel = emdata.groupby('cluster_reference')['prefire_fuel'].median().to_frame(name = 'prefire_fuel').reset_index()
prefirefuel.head()

,cluster_reference,prefire_fuel
0,2003_0,337.042602
1,2003_1,6220.097576
2,2003_10,162.188442
3,2003_100,3390.377328
4,2003_1000,2639.337818


In [23]:
fuel_moisture_class = emdata.groupby('cluster_reference')['fuel_moisture_class'].agg(pd.Series.mode).to_frame(name = 'fuel_moisture_class').reset_index()
fuel_moisture_class.head()

,cluster_reference,fuel_moisture_class
0,2003_0,2
1,2003_1,4
2,2003_10,2
3,2003_100,2
4,2003_1000,3


In [55]:
fuel_moisture_class['fuel_moisture_class2'] = fuel_moisture_class.apply(lambda x: (x['fuel_moisture_class'].max()),axis=1)

In [27]:
NASA_M6_Model = NASA_M6_Model.rename(columns={'target_clusterref':'cluster_reference'})
NASA_M6_Model.head()

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,datetime,doy,month,year,cluster_reference,season
0,0,38.8142,-93.5539,300.8,1.0,1.0,2003-01-01,423,Terra,MODIS,33,6.2,267.0,10.4,N,0,2003-01-01,1,1,2003,2003_1980,3
21,21,34.5954,-78.6218,306.5,1.3,1.1,2003-01-01,1624,Terra,MODIS,64,6.2,289.2,11.0,D,0,2003-01-01,1,1,2003,2003_4279,3
22,22,33.4182,-110.8618,307.6,1.2,1.1,2003-01-01,1803,Terra,MODIS,66,6.2,285.1,10.8,D,2,2003-01-01,1,1,2003,2003_1522,3
23,23,29.7120,-95.1284,307.2,1.0,1.0,2003-01-01,1934,Aqua,MODIS,65,6.2,294.1,5.6,D,0,2003-01-01,1,1,2003,2003_919,3
24,24,28.9161,-98.6293,313.3,1.3,1.1,2003-01-01,1934,Aqua,MODIS,69,6.2,297.4,12.0,D,0,2003-01-01,1,1,2003,2003_777,3


--- 

In [58]:
# Merging data into NASA M6.
NASA_M6_Model = NASA_M6_Model.merge(right=covertypedf.loc[:,['cluster_reference', 'covertype2']],
                   how='left',
                   left_on=['cluster_reference'],
                   right_on=['cluster_reference'])


In [40]:
NASA_M6_Model.head(20)

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,datetime,doy,month,year,cluster_reference,season,covertype,prefirefuel,prefire_fuel_x,fuel_moisture_class_x,fuelcode,prefire_fuel_y,fuel_moisture_class_y
0,0,38.8142,-93.5539,300.8,1.0,1.0,2003-01-01,423,Terra,MODIS,33,6.2,267.0,10.4,N,0,2003-01-01,1,1,2003,2003_1980,3,"[1, 3]",NaN,NaN,NaN,"[1, 2700]",1439.152585,2
1,21,34.5954,-78.6218,306.5,1.3,1.1,2003-01-01,1624,Terra,MODIS,64,6.2,289.2,11.0,D,0,2003-01-01,1,1,2003,2003_4279,3,3,NaN,NaN,NaN,1600,6220.097576,3
2,22,33.4182,-110.8618,307.6,1.2,1.1,2003-01-01,1803,Terra,MODIS,66,6.2,285.1,10.8,D,2,2003-01-01,1,1,2003,2003_1522,3,3,NaN,NaN,NaN,1220,4534.187262,2
3,23,29.7120,-95.1284,307.2,1.0,1.0,2003-01-01,1934,Aqua,MODIS,65,6.2,294.1,5.6,D,0,2003-01-01,1,1,2003,2003_919,3,1,NaN,NaN,NaN,1,277.412850,2
4,24,28.9161,-98.6293,313.3,1.3,1.1,2003-01-01,1934,Aqua,MODIS,69,6.2,297.4,12.0,D,0,2003-01-01,1,1,2003,2003_777,3,1,NaN,NaN,NaN,1,251.296812,2
5,27,33.1976,-100.2702,302.6,1.3,1.1,2003-01-01,1935,Aqua,MODIS,27,6.2,288.5,7.1,D,0,2003-01-01,1,1,2003,2003_1370,3,2,NaN,NaN,NaN,2,750.808071,2
6,28,31.9433,-98.9875,304.5,1.2,1.1,2003-01-01,1935,Aqua,MODIS,59,6.2,290.7,7.2,D,0,2003-01-01,1,1,2003,2003_1278,3,2,NaN,NaN,NaN,2,470.424942,2
7,29,32.7772,-95.0444,301.3,1.0,1.0,2003-01-01,1935,Aqua,MODIS,46,6.2,289.9,4.2,D,0,2003-01-01,1,1,2003,2003_3100,3,1,NaN,NaN,NaN,1,173.172870,2
8,30,32.2024,-94.6048,302.1,1.0,1.0,2003-01-01,1935,Aqua,MODIS,44,6.2,288.5,5.1,D,0,2003-01-01,1,1,2003,2003_3015,3,3,NaN,NaN,NaN,"[1160, 1400]",6935.101645,2
9,31,33.3287,-95.5265,317.5,1.0,1.0,2003-01-01,1935,Aqua,MODIS,78,6.2,288.3,15.9,D,0,2003-01-01,1,1,2003,2003_1290,3,1,NaN,NaN,NaN,1,393.589989,2


In [43]:
NASA_M6_Model = NASA_M6_Model.drop(columns=['Unnamed: 0','scan', 'track', 'acq_date', 'acq_date', 'acq_date', 'satellite', 'instrument',
                                            'confidence', 'version', 'type', 'datetime', 'prefirefuel','prefire_fuel_x','fuel_moisture_class_x'])

In [60]:
NASA_M6_Model = NASA_M6_Model.drop(columns=['covertype', 'fuelcode', 'fuel_moisture_class_y'])

In [67]:
NASA_M6_Model = NASA_M6_Model.rename(columns={'prefire_fuel_':'prefire_fuel'})

In [68]:
NASA_M6_Model.head(20)

,latitude,longitude,brightness,acq_time,bright_t31,frp,daynight,doy,month,year,cluster_reference,season,prefire_fuel,fuel_moisture_class,fuelcode,covertype
0,38.8142,-93.5539,300.8,423,267.0,10.4,N,1,1,2003,2003_1980,3,1439.152585,2,2700,3
1,34.5954,-78.6218,306.5,1624,289.2,11.0,D,1,1,2003,2003_4279,3,6220.097576,3,1600,3
2,33.4182,-110.8618,307.6,1803,285.1,10.8,D,1,1,2003,2003_1522,3,4534.187262,2,1220,3
3,29.7120,-95.1284,307.2,1934,294.1,5.6,D,1,1,2003,2003_919,3,277.412850,2,1,1
4,28.9161,-98.6293,313.3,1934,297.4,12.0,D,1,1,2003,2003_777,3,251.296812,2,1,1
5,33.1976,-100.2702,302.6,1935,288.5,7.1,D,1,1,2003,2003_1370,3,750.808071,2,2,2
6,31.9433,-98.9875,304.5,1935,290.7,7.2,D,1,1,2003,2003_1278,3,470.424942,2,2,2
7,32.7772,-95.0444,301.3,1935,289.9,4.2,D,1,1,2003,2003_3100,3,173.172870,2,1,1
8,32.2024,-94.6048,302.1,1935,288.5,5.1,D,1,1,2003,2003_3015,3,6935.101645,2,1400,3
9,33.3287,-95.5265,317.5,1935,288.3,15.9,D,1,1,2003,2003_1290,3,393.589989,2,1,1


In [69]:
NASA_M6_Model = NASA_M6_Model[['latitude', 'longitude', 'doy', 'month', 'year','cluster_reference', 'season', 'covertype', 'fuelcode', 'prefire_fuel','fuel_moisture_class',
                               'brightness', 'bright_t31', 'frp']]

In [71]:
print(NASA_M6_Model.shape)
NASA_M6_Model.head()

(1352421, 14)


,latitude,longitude,doy,month,year,cluster_reference,season,covertype,fuelcode,prefire_fuel,fuel_moisture_class,brightness,bright_t31,frp
0,38.8142,-93.5539,1,1,2003,2003_1980,3,3,2700,1439.152585,2,300.8,267.0,10.4
1,34.5954,-78.6218,1,1,2003,2003_4279,3,3,1600,6220.097576,3,306.5,289.2,11.0
2,33.4182,-110.8618,1,1,2003,2003_1522,3,3,1220,4534.187262,2,307.6,285.1,10.8
3,29.7120,-95.1284,1,1,2003,2003_919,3,1,1,277.412850,2,307.2,294.1,5.6
4,28.9161,-98.6293,1,1,2003,2003_777,3,1,1,251.296812,2,313.3,297.4,12.0


In [72]:
NASA_M6_Model.to_csv('/Users/AlfHaugen/Python/Wildfire_Data/FireExports/FireIntensity_Model_June1.csv',index=False)
                     

----

In [10]:
query1 = """
SELECT *
FROM Weather;
"""

In [13]:
weatherdf = pd.read_sql(query1, engine)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'db27bvsdruzh45.Weather' doesn't exist
[SQL: 
SELECT *
FROM Weather;
]
(Background on this error at: http://sqlalche.me/e/f405)